# 初始化

In [2]:
import os
from local_packages import *                                                                                                                                                                                                                                                                                                                                               
from dotenv import load_dotenv
import numpy as np

# 加载环境变量
dotenv_path = os.path.join(os.getcwd(), '.env')

# 设置项目根目录
project_root = os.path.dirname(dotenv_path)

# 初始化API
agentopener=AgentOpener(service_type='qwen', version='long')

llm=agentopener.service

# 初始化向量嵌入
embedder=QwenEmbedder()

# 初始化文字分割器
divider=TextDivider(threshold=4096,overlap=512)
processor=LLMParser()

服务初始化成功


In [ ]:
input_string=r'''
宴桃园豪杰三结义　斩黄巾英雄首立功

　　滚滚长江东逝水，浪花淘尽英雄。是非成败转头空。青山依旧在，几度夕阳红。

　　白发渔樵江渚上，惯看秋月春风。一壶浊酒喜相逢。古今多少事，都付笑谈中。

　　——调寄《临江仙》

　　话说天下大势，分久必合，合久必分。周末七国分争，并入于秦。及秦灭之后，楚、汉分争，又并入于汉。汉朝自高祖斩白蛇而起义，一统天下，后来光武中兴，传至献帝，遂分为三国。推其致乱之由，殆始于桓、灵二帝。桓帝禁锢善类，崇信宦官。及桓帝崩，灵帝即位，大将军窦武、太傅陈蕃，共相辅佐。时有宦官曹节等弄权，窦武、陈蕃谋诛之，机事不密，反为所害，中涓自此愈横。

　　建宁二年四月望日，帝御温德殿。方升座，殿角狂风骤起。只见一条大青蛇，从梁上飞将下来，蟠于椅上。帝惊倒，左右急救入宫，百官俱奔避。须臾，蛇不见了。忽然大雷大雨，加以冰雹，落到半夜方止，坏却房屋无数。建宁四年二月，洛阳地震；又海水泛溢，沿海居民，尽被大浪卷入海中。光和元年，雌鸡化雄。六月朔，黑气十余丈，飞入温雄殿中。秋七月，有虹现于玉堂；五原山岸，尽皆崩裂。种种不祥，非止一端。帝下诏问群臣以灾异之由，议郎蔡邕上疏，以为蜺堕鸡化，乃妇寺干政之所致，言颇切直。帝览奏叹息，因起更衣。曹节在后窃视，悉宣告左右；遂以他事陷邕于罪，放归田里。后张让、赵忠、封谞、段珪、曹节、侯览、蹇硕、程旷、夏惲、郭胜十人朋比为奸，号为“十常侍”。帝尊信张让，呼为“阿父”。朝政日非，以致天下人心思乱，盗贼蜂起。

　　时巨鹿郡有兄弟三人，一名张角，一名张宝，一名张梁。那张角本是个不第秀才，因入山采药，遇一老人，碧眼童颜，手执藜杖，唤角至一洞中，以天书三卷授之，曰：“此名《太平要术》，汝得之，当代天宣化，普救世人；若萌异心，必获恶报。”角拜问姓名。老人曰：“吾乃南华老仙也。”言讫，化阵清风而去。角得此书，晓夜攻习，能呼风唤雨，号为“太平道人”。中平元年正月内，疫气流行，张角散施符水，为人治病，自称“大贤良师”。角有徒弟五百余人，云游四方，皆能书符念咒。次后徒众日多，角乃立三十六方，大方万余人，小方六七千，各立渠帅，称为将军；讹言：“苍天已死，黄天当立；岁在甲子，天下大吉。”令人各以白土，书“甲子”二字于家中大门上。青、幽、徐、冀、荆、扬、兖、豫八州之人，家家侍奉大贤良师张角名字。角遣其党马元义，暗赍金帛，结交中涓封谞，以为内应。角与二弟商议曰：“至难得者，民心也。今民心已顺，若不乘势取天下，诚为可惜。”遂一面私造黄旗，约期举事；一面使弟子唐周，驰书报封谞。唐周乃径赴省中告变。帝召大将军何进调兵擒马元义，斩之；次收封谞等一干人下狱。张角闻知事露，星夜举兵，自称“天公将军”，张宝称“地公将军”，张梁称“人公将军”。申言于众曰：“今汉运将终，大圣人出。汝等皆宜顺天从正，以乐太平。”四方百姓，裹黄巾从张角反者四五十万。贼势浩大，官军望风而靡。何进奏帝火速降诏，令各处备御，讨贼立功。
'''

data_template01_community_label_dict=f'''
{{
    '标签1': score_float1,
    ...
    '标签n': score_floatn,
}}
'''

prompt01_community_label_dict=f'''
你是一个聪明的助手，你的工作是帮助人们理解文本中的信息。
当给定一段文字时，你的任务是为这段文字分配一个描述性的领域标签，用来概括文本的主要内容。
例如，可能的领域有：“社会学”，“算法分析”，“医学科学”等。同时你也会为文本和这个标签的相关性程度打分，分数是0到10之间的浮点数，1表示微不足道或不相关，10表示具有高度和深刻的相关性
接下来是正式的任务：
--------------------------------------------------------------------------------
请根据给定的文本，选择几个最合适的领域标签。
对于这一段文字:

{input_string}

我希望你识别出它所涉及到的主题，并且采用下面的json dict格式回复：

{data_template01_community_label_dict}
'''

community_label_answer=llm.ask(prompt01_community_label_dict)

community_label_dict=processor.parse_dict(community_label_answer)

def validation01_community_label_dict(data):
    """
    检查数据是否符合{string: float, ..., string: float}的格式。

    参数:
        data (dict): 待检查的数据。

    返回:
        bool: 如果数据符合格式则返回True，否则返回False。
    """
    if not isinstance(data, dict):
        return False
    
    for key, value in data.items():
        if not isinstance(key, str):
            return False
        if not isinstance(value, float):
            return False
    
    return True

correction01_community_label_dict=f'''
下列内容中含有一个错误的数据格式：

{community_label_answer}

请你修改它，使其符合以下格式：

{data_template01_community_label_dict}
'''
print(community_label_dict)

In [ ]:
data_template02_entity_type_list=f'''
['实体类型1',...,'实体类型n']
'''

prompt02_entity_type_list=f'''
你是一个聪明的助手，你的工作是帮助人们理解文本中的信息。
当给定一段文字时，你的任务是为这段文字中的实体进行分类。
非常重要：不要生成重复或重叠的实体类型。例如，如果文本包含"公司"和"组织"，你应该只返回其中一个。
数量不是重点，质量优先。确保你的答案中的每一个内容都与实体提取的上下文相关。
记住，我们需要的是实体类型。
接下来是正式的任务：
--------------------------------------------------------------------------------
请根据给定的文本，给出其中涉及的实体类型。
对于这一段文字:

{input_string}

我希望你识别出它所涉及到的主题，并且采用下面的格式回复：

{data_template02_entity_type_list}
'''

entity_type_answer=llm.ask(prompt02_entity_type_list)

entity_type_list=processor.parse_list(entity_type_answer)

def validation02_entity_type_list(data):
    """
    检查数据是否符合[string, ..., string]的格式。

    参数:
        data (list): 待检查的数据。

    返回:
        bool: 如果数据符合格式则返回True，否则返回False。
    """
    if not isinstance(data, list):
        return False
    
    for item in data:
        if not isinstance(item, str):
            return False
    
    return True

correction02_entity_type_list=f'''
下列内容中含有一个错误的数据格式：

{entity_type_answer}

请你修改它，使其符合以下格式：

{data_template02_entity_type_list}
'''


In [ ]:
data_template03_entity_extract=f'''
{{
    '实体名称1': {{'相关性分数':score_float1,'实体类型':'type_name_str'}},
    ...,
    '实体名称n': {{'相关性分数':score_floatn,'实体类型':'type_name_str'}}
}}
'''

prompt03_entity_extract=f'''
你是一个聪明的助手，你的工作是帮助人们理解文本中的信息。
给定一个可能与这个活动相关的文本和，请你找出文本中的所有{entity_type_list[1]}类型的实体。
请你找出所有实体。对每个找到的实体，提取以下信息：
   - 实体名称：原文中的实体名称
   - 实体类型：以下类型之一：{entity_type_list[1]}
   - 贴切程度分数：关于实体和{entity_type_list[1]}类型之间的相关性打分，分数是0到10之间的浮点数，1表示微不足道或不相关，10表示具有高度和深刻的相关性
接下来是正式的任务：
--------------------------------------------------------------------------------
请根据给定的文本，给出其中涉及的实体类型。
对于这一段文字:

{input_string}

我希望你抽取出它其中的所有所有实体，并且采用下面的json dict格式回复：

{data_template03_entity_extract}
'''

entity_extract_answer=llm.ask(prompt03_entity_extract)

entity_extract_dict=processor.parse_dict(entity_extract_answer)

def validation03_entity_extract(data):
    """
    检查数据是否符合
    {string: {'相关性分数': float, '实体类型': string}, ..., string: {'相关性分数': float, '实体类型': string}}
    的格式。

    参数:
        data (dict): 待检查的数据。

    返回:
        bool: 如果数据符合格式则返回True，否则返回False。
    """
    if not isinstance(data, dict):
        return False
    
    for key, value in data.items():
        if not isinstance(key, str):
            return False
        if not isinstance(value, dict):
            return False
        if '相关性分数' not in value or '实体类型' not in value:
            return False
        if not isinstance(value['相关性分数'], float):
            return False
        if not isinstance(value['实体类型'], str):
            return False
    
    return True

correction03_entity_extract=f'''
下列内容中含有一个错误的数据格式：

{entity_extract_answer}

请你修改它，使其符合以下格式：

{data_template03_entity_extract}
'''


In [ ]:
import threading
from queue import Queue

def summarize(input_text, concern_topic=None):
    summarize_prompt = f'''
    你是一个出色的文字压缩器，以下有一段长文本，我期待你对其进行压缩和摘要：

    {input_text}

    请你不要输出任何无关内容，仅输出压缩后的文本。
    '''
    if concern_topic:
        summarize_prompt += f'''
        我还期望你仅输出与{concern_topic}相关的内容。
        '''
    return llm.ask(summarize_prompt)

def summarize_worker(queue, output, concern_topic):
    while not queue.empty():
        index, input_text = queue.get()
        summarized_text = summarize(input_text, concern_topic)
        output[index] = summarized_text
        queue.task_done()

def summarize_list(string_list, thread_number, concern_topic=None):
    queue = Queue()
    output = [None] * len(string_list)
    
    # 将每个字符串及其索引放入队列
    for i, input_text in enumerate(string_list):
        queue.put((i, input_text))
    
    threads = []
    # 创建指定数量的线程来处理队列中的任务
    for _ in range(thread_number):
        thread = threading.Thread(target=summarize_worker, args=(queue, output, concern_topic))
        thread.start()
        threads.append(thread)
    
    # 等待所有任务完成
    queue.join()
    
    # 确保所有线程都已完成
    for thread in threads:
        thread.join()
    
    return output


